In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=59a6274a4e21906e1d136addfd13fc5bf32953687020942822d47ce7892db04d
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import rand, col
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import StructType, StructField
from pyspark.sql import functions as F
from pyspark.sql.types import NumericType
import pickle
import time
from pyspark.ml.regression import DecisionTreeRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import isnan, when, count, isnull, avg, expr, min, max, col, rand
from pyspark.sql.functions import col, to_date, month, year
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import regexp_replace, struct, udf
from pyspark.sql.functions import regexp_extract

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder \
    .appName("Bank Project") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35") \
    .getOrCreate()
spark.conf.set("spark.sql.codegen.wholeStage", "false")

In [ ]:
data = spark.read.csv('/content/drive/MyDrive/Bank Project/Loan_Data.csv', header=True, inferSchema=True)

# **Drop duplicate rows to avoid data leakage and then save train and test separately.**

In [ ]:
data.count()

2925493

In [ ]:
data = data.drop_duplicates(data.columns[2:])

In [ ]:
data.count()

2925493

In [ ]:
train, test = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
train.write.csv('/content/drive/MyDrive/Bank Project/train', header=True)
test.write.csv('/content/drive/MyDrive/Bank Project/test', header=True)

# **Read train data**

In [ ]:
train = spark.read.csv('/content/drive/MyDrive/Bank Project/train', header=True, inferSchema=True)

In [ ]:
train.show(5)

+---+---------+---------+-----------+---------------+---------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+---------------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+--

# **Drop all unnecessary columns**

In [ ]:
def drop_clear_cols(data):
  return data.drop('_c0', 'id', 'url', 'zip_code', 'grade', 'title')

# **Drop all columns which contain more than 90 % constant values.**

In [ ]:
def drop_unnecessary_columns_by_constants(data, threshold=0.9):
    row_num = data.count()

    for col_name in data.columns:
        value_counts = data.groupBy(col_name).count()

        max_count_row = value_counts.agg({"count": "max"}).collect()[0]
        max_count = max_count_row[0]

        max_count = float(max_count)

        if max_count / row_num > threshold:
            data = data.drop(col_name)
    return data

In [ ]:
def all_drop_columns(data):
  data = drop_unnecessary_columns_by_constants(drop_clear_cols(data))
  return data

In [ ]:
train = all_drop_columns(train)

# **Create columnset which is proper for training and testing**

In [ ]:
with open('/content/drive/MyDrive/Bank Project/columns.pkl', 'wb') as file:
    pickle.dump(train.columns, file)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Bank Project/columns.pkl', 'rb') as file:
    columnset = pickle.load(file)

In [ ]:
train = spark.read.csv('/content/drive/MyDrive/Bank Project/train', header=True, inferSchema=True)

In [ ]:
final_cols = []
for i in train.columns:
  if i in columnset and i not in ['acc_now_delinq', 'delinq_2yrs', 'delinq_amnt', 'mths_since_recent_revol_delinq', 'mths_since_recent_bc_dlq', 'mths_since_last_delinq', 'pct_tl_nvr_dlq']:
    final_cols.append(i)

In [ ]:
with open('/content/drive/MyDrive/Bank Project/columns_.pkl', 'wb') as file:
    pickle.dump(final_cols + ['acc_now_delinq', 'delinq_2yrs', 'delinq_amnt'], file)

In [ ]:
with open('/content/drive/MyDrive/Bank Project/columns_.pkl', 'rb') as file:
    final_cols = pickle.load(file)
train = spark.read.csv('/content/drive/MyDrive/Bank Project/train', header=True, inferSchema=True)

In [ ]:
train = train.select(*final_cols)

In [ ]:
train.show()

+---------+-----------+---------------+---------+--------+-----------+---------+--------------------+----------+--------------+----------+-------------------+--------+---------------+------------------+----------+-----+----------------+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------+---------------+------------+------------------+--------------------+-------------------+---------------------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------+--------------------+---------------------+--------------+--------+--------------------+---------------------+-------------------

# **Drop rows which contain 80% and more null values.**

In [ ]:
def drop_rows(data):
  def null_percentage(row):
    null_count = sum([1 for val in row if val is None])
    total_count = len(row)
    return (null_count / total_count) * 100

  null_percentage_udf = udf(null_percentage, FloatType())

  columns = data.columns
  df = data.withColumn("null_percentage", null_percentage_udf(struct(*columns)))

  filtered_df = df.filter(col("null_percentage") <= 80)

  filtered_df = filtered_df.drop("null_percentage")
  return filtered_df

# **Convert categorical and nonnumerical data into numeric**

In [ ]:
def process_term(data):
    data = data.withColumn('term', regexp_extract(col('term'), r'(\d+)', 1))
    return data

In [ ]:
def process_int_rate(data):
    data = data.withColumn('int_rate', regexp_replace(col('int_rate'), '%', ''))
    return data

In [ ]:
def process_revol_util(data):
    data = data.withColumn('revol_util', regexp_replace(col('int_rate'), '%', ''))
    return data

In [ ]:
def process_emp_length(data):
    data = data.withColumn('emp_length', regexp_replace(col('emp_length'), r'\+', ''))

    data = data.withColumn('emp_length', regexp_extract(col('emp_length'), r'(\d+)', 1))

    data = data.withColumn('emp_length', col('emp_length').cast('int'))

    return data

In [ ]:
def process_grade(data):
    def map_sub_grade(sub_grade):
        mapping = {
            'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4,
            'B1': 5, 'B2': 6, 'B3': 7, 'B4': 8, 'B5': 9,
            'C1': 10, 'C2': 11, 'C3': 12, 'C4': 13, 'C5': 14,
            'D1': 15, 'D2': 16, 'D3': 17, 'D4': 18, 'D5': 19,
            'E1': 20, 'E2': 21, 'E3': 22, 'E4': 23, 'E5': 24,
            'F1': 25, 'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29,
            'G1': 30, 'G2': 31, 'G3': 32, 'G4': 33, 'G5': 34
        }
        return mapping.get(sub_grade, -1)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('sub_grade', map_sub_grade_udf(col('sub_grade')))
    return df_encoded

In [ ]:
def process_emp_title(data):
  def categorize_profession(title):
      if not title:
          return 'Unknown'
      title = title.lower()
      if any(keyword in title for keyword in ['manager', 'director', 'vp', 'executive', 'head', 'chief', 'president', 'supervisor', 'coordinator']):
          return 'Management and Executive Roles'
      elif any(keyword in title for keyword in ['assistant', 'coordinator', 'clerk', 'office', 'receptionist', 'secretary', 'administrator', 'data entry', 'support', 'specialist', 'scheduler']):
          return 'Administrative and Support Roles'
      elif any(keyword in title for keyword in ['engineer', 'technician', 'developer', 'it', 'analyst', 'architect', 'programmer', 'consultant', 'tech', 'network', 'systems', 'software', 'hardware']):
          return 'Technical and Engineering Roles'
      elif any(keyword in title for keyword in ['nurse', 'therapist', 'counselor', 'healthcare', 'clinician', 'practitioner', 'physician', 'medical', 'pharmacist', 'social worker', 'aide', 'caregiver']):
          return 'Healthcare and Social Services Roles'
      elif any(keyword in title for keyword in ['sales', 'customer service', 'representative', 'associate', 'agent', 'account manager', 'client service', 'business development', 'account executive']):
          return 'Sales and Customer Service Roles'
      elif any(keyword in title for keyword in ['teacher', 'instructor', 'professor', 'lecturer', 'tutor', 'educator', 'trainer', 'academic advisor']):
          return 'Education and Training Roles'
      elif any(keyword in title for keyword in ['technician', 'mechanic', 'electrician', 'plumber', 'carpenter', 'welder', 'machinist', 'laborer', 'foreman', 'operator']):
          return 'Skilled Trades and Labor'
      elif any(keyword in title for keyword in ['designer', 'artist', 'graphic designer', 'creative director', 'art director', 'web designer', 'illustrator', 'photographer', 'stylist']):
          return 'Creative and Design Roles'
      elif any(keyword in title for keyword in ['accountant', 'auditor', 'financial analyst', 'controller', 'bookkeeper', 'tax preparer', 'finance manager', 'investment analyst']):
          return 'Finance and Accounting Roles'
      elif any(keyword in title for keyword in ['attorney', 'lawyer', 'paralegal', 'legal assistant', 'compliance officer', 'legal advisor', 'legal secretary']):
          return 'Legal and Compliance Roles'
      else:
          return 'Unknown'

  categorize_profession_udf = udf(categorize_profession, StringType())

  df_categorized = data.withColumn('emp_title', categorize_profession_udf(col('emp_title')))

  indexer = StringIndexer(inputCol='emp_title', outputCol='emp_title_index')
  df_indexed = indexer.fit(df_categorized).transform(df_categorized)

  encoder = OneHotEncoder(inputCols=['emp_title_index'], outputCols=['emp_title_encoded'])
  df_encoded = encoder.fit(df_indexed).transform(df_indexed)
  df_encoded = df_encoded.drop('emp_title_index')
  df_encoded = df_encoded.drop('emp_title')
  return df_encoded

In [ ]:
def process_home_ownership(data):
    def map_sub_grade(value):
        mapping = {
            'RENT': 0,
            'MORTGAGE': 1,
            'OWN': 2,
        }
        return mapping.get(value, 3)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('home_ownership', map_sub_grade_udf(col('home_ownership')))

    one_hot_encoder = OneHotEncoder(inputCols=['home_ownership'], outputCols=['home_ownership_encoded'])
    df_encoded = one_hot_encoder.fit(df_encoded).transform(df_encoded)
    df_encoded = df_encoded.drop('home_ownership')
    return df_encoded

In [ ]:
def process_verification_status(data):
    def map_sub_grade(value):
        mapping = {
            'Verified': 0,
            'Source Verified': 1,
            'Not Verified': 2,
        }
        return mapping.get(value, 3)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('verification_status', map_sub_grade_udf(col('verification_status')))

    one_hot_encoder = OneHotEncoder(inputCols=['verification_status'], outputCols=['verification_status_encoded'])
    df_encoded = one_hot_encoder.fit(df_encoded).transform(df_encoded)
    df_encoded = df_encoded.drop('verification_status')
    return df_encoded

In [ ]:
def process_loan_status(data):
    def map_sub_grade(value):
        mapping = {
            'Fully Paid': 0,
            'Charged Off': 1,
            'Current': 2,
            'In Grace Period': 3,
            'Late (16-30 days)': 4,
            'Late (31-120 days)': 5,
        }
        return mapping.get(value, 6)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('loan_status', map_sub_grade_udf(col('loan_status')))

    one_hot_encoder = OneHotEncoder(inputCols=['loan_status'], outputCols=['loan_status_encoded'])
    df_encoded = one_hot_encoder.fit(df_encoded).transform(df_encoded)
    df_encoded = df_encoded.drop('loan_status')
    return df_encoded

In [ ]:
def process_purpose(data):
    def map_sub_grade(value):
        mapping = {
            'wedding': 0,
            'educational': 1,
            'other': 2,
            'small_business': 3,
            'debt_consolidation': 4,
            'credit_card': 5,
            'moving': 6,
            'vacation': 7,
            'renewable_energy': 8,
            'house': 9,
            'car': 10,
            'major_purchase': 11,
            'medical': 12,
            'home_improvement': 13
        }
        return mapping.get(value, 14)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('purpose', map_sub_grade_udf(col('purpose')))

    one_hot_encoder = OneHotEncoder(inputCols=['purpose'], outputCols=['purpose_encoded'])
    df_encoded = one_hot_encoder.fit(df_encoded).transform(df_encoded)
    df_encoded = df_encoded.drop('purpose')
    return df_encoded

In [ ]:
def process_addr_state(data):
    def map_sub_grade(value):
        mapping = {
        'AK': 0,
        'AL': 1,
        'AR': 2,
        'AZ': 3,
        'CA': 4,
        'CO': 5,
        'CT': 6,
        'DE': 7,
        'FL': 8,
        'GA': 9,
        'HI': 10,
        'IA': 11,
        'ID': 12,
        'IL': 13,
        'IN': 14,
        'KS': 15,
        'KY': 16,
        'LA': 17,
        'MA': 18,
        'MD': 19,
        'ME': 20,
        'MI': 21,
        'MN': 22,
        'MO': 23,
        'MS': 24,
        'MT': 25,
        'NC': 26,
        'ND': 27,
        'NE': 28,
        'NH': 29,
        'NJ': 30,
        'NM': 31,
        'NV': 32,
        'NY': 33,
        'OH': 34,
        'OK': 35,
        'OR': 36,
        'PA': 37,
        'RI': 38,
        'SC': 39,
        'SD': 40,
        'TN': 41,
        'TX': 42,
        'UT': 43,
        'VA': 44,
        'VT': 45,
        'WA': 46,
        'WI': 47,
        'WV': 48,
        'WY': 49}
        return mapping.get(value, 50)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('addr_state', map_sub_grade_udf(col('addr_state')))

    return df_encoded

In [ ]:
def process_initial_list_status(data):
    def map_sub_grade(value):
        mapping = {
            'f': 0,
            'w': 1,
        }
        return mapping.get(value, 2)

    map_sub_grade_udf = udf(map_sub_grade, IntegerType())

    df_encoded = data.withColumn('initial_list_status', map_sub_grade_udf(col('initial_list_status')))

    one_hot_encoder = OneHotEncoder(inputCols=['initial_list_status'], outputCols=['initial_list_status_encoded'])
    df_encoded = one_hot_encoder.fit(df_encoded).transform(df_encoded)
    df_encoded = df_encoded.drop('initial_list_status')
    return df_encoded

In [ ]:
def from_time_to_numeric(data):
  lst = ['earliest_cr_line', 'issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']
  input_date_format = "MMM-yyyy"
  for i in lst:
      data = data.withColumn("date", to_date(col(i), input_date_format))

      data = data.withColumn(f"month_{i}", month(col("date")))
      data = data.withColumn(f"year_{i}", year(col("date")))
      data = data.drop('date', i)
  return data

In [ ]:
def process_delinq_2yrs(data):
    def map_delinq_2yrs(delinq_2yrs):
        mapping = {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: 3
        }
        return mapping.get(delinq_2yrs, 4)

    map_sub_grade_udf = udf(map_delinq_2yrs, IntegerType())

    df_encoded = data.withColumn('delinq_2yrs', map_sub_grade_udf(col('delinq_2yrs')))
    return df_encoded

In [ ]:
def process_acc_now_delinq(data):
    def map_acc_now_delinq(acc_now_delinq):
        mapping = {
            0: 0
        }
        return mapping.get(acc_now_delinq, 1)

    map_sub_grade_udf = udf(map_acc_now_delinq, IntegerType())

    df_encoded = data.withColumn('acc_now_delinq', map_sub_grade_udf(col('acc_now_delinq')))
    return df_encoded

In [ ]:
def cast_int(data):
  for i in data.columns:
    if data.select(i).dtypes[0][1] == 'string':
        data = data.withColumn(i, col(i).cast('integer'))
  return data

# **Handle all above transformations.**

In [ ]:
train = drop_rows(train)
train = process_term(train)
train = process_int_rate(train)
train = process_revol_util(train)
train = process_emp_length(train)
train = process_grade(train)
train = process_emp_title(train)
train = process_home_ownership(train)
train = process_verification_status(train)
train = process_loan_status(train)
train = process_purpose(train)
train = process_addr_state(train)
train = process_initial_list_status(train)
train = from_time_to_numeric(train)
train = process_delinq_2yrs(train)
train = process_acc_now_delinq(train)
train = cast_int(train)

# **Calculate how many null values are in each column.**

In [ ]:
def calculate_nulls(data):
    vector_columns = [c for c in data.columns if isinstance(data.schema[c].dataType, VectorUDT)]
    null_counts = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns if c not in vector_columns])
    return null_counts

# **Save train parameters: median, min, max.**

In [ ]:
def save_train_parameters(data: DataFrame, output_path: str) -> DataFrame:
    vector_columns = [c for c in data.columns if isinstance(data.schema[c].dataType, VectorUDT)]
    non_vector_columns = [c for c in data.columns if c not in vector_columns]

    stats_list = []
    for col_name in non_vector_columns:
        stats = data.select(
            expr(f'percentile_approx({col_name}, 0.5)').cast(DoubleType()).alias('median'),
            min(col_name).cast(DoubleType()).alias('min'),
            max(col_name).cast(DoubleType()).alias('max')
        ).collect()[0]

        stats_list.append((
            col_name,
            stats['median'],
            stats['min'],
            stats['max']
        ))
        print(f'Column {col_name} processed')

    stats_df = data.sparkSession.createDataFrame(
        stats_list,
        ["column_name", "median", "min", "max"]
    )

    stats_df.show()

    stats_df.write.csv(output_path, header=True, mode = 'overwrite')

    return stats_df


In [ ]:
save_train_parameters(train, '/content/drive/MyDrive/Bank Project/train_parameters')

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column emp_length processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_low processed
Column fico_range_high processed
Column inq_last_6mths processed
Column mths_since_last_record processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column mths_since_last_major_derog processed
Column tot_coll_amt processed
Column tot_cur_bal processed
Column open_acc_6m processed
Column 

DataFrame[column_name: string, median: double, min: double, max: double]

# **Save data which has all columns integer or vector.**

In [ ]:
train.write.parquet('/content/drive/MyDrive/Bank Project/half_processed_train', mode='overwrite')

In [ ]:
train = spark.read.parquet('/content/drive/MyDrive/Bank Project/half_processed_train')

In [ ]:
train.show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----------+----------+---+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+---------+-------------+----------------+---------------+---------------+-------------+---------------+--------------------+-------------------+---------------------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------+--------------------+---------------------+--------------+--------+--------------------+---------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+------------------+----------------+--------------------+---------

# **How many null values are in each column.**

In [ ]:
calculate_nulls(train).show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----------+----------+----+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+---------------+--------------------+-------------------+---------------------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------+--------------------+---------------------+--------------+--------+--------------------+---------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+------------------+----------------+--------------------+-------------

# **Fill missing values for train data**

In [ ]:
target = ['acc_now_delinq', 'acc_now_delinq', 'delinq_amnt']

# **Drop all null values which are in target.**

In [ ]:
train = train.dropna(subset=target)

In [ ]:
calculate_nulls(train).show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----------+----------+----+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+---------------+--------------------+-------------------+---------------------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------+--------------------+---------------------+--------------+--------+--------------------+---------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+------------------+----------------+--------------------+-------------

# **Determine which columns' null values must be replaced with medians and which must be predicted by models.**

In [ ]:
medians = train.columns[:7] + train.columns[8:14] + train.columns[15:29] + train.columns[-10:-4] + train.columns[-2:]
models = train.columns[7:8] + train.columns[14:15] + train.columns[29:-19] + train.columns[-4:-2]

# **Replace some columns by medians.**

In [ ]:
def replace_nulls_with_medians(data, columnset, train_parameters):
    for col_name in columnset:
        median_value = train_parameters.filter(col('column_name') == col_name).select('median').collect()[0][0]

        data = data.withColumn(col_name, when(col(col_name).isNull(), median_value).otherwise(col(col_name)))

    return data

In [ ]:
parameters = spark.read.csv('/content/drive/MyDrive/Bank Project/train_parameters', header=True, inferSchema=True)

In [ ]:
train = replace_nulls_with_medians(train, medians, parameters)

In [ ]:
calculate_nulls(train).show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----------+----------+---+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+---------------+--------------------+-------------------+---------------------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------+--------------------+---------------------+--------------+--------+--------------------+---------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+------------------+----------------+--------------------+--------------

# **Train models for left columns with null values.**

In [ ]:
def save_models_without_filling(data: DataFrame, columnset1: list, columnset2: list, save_path: str, test_ratio: float = 0.1):
    accuracies = {}

    for col2 in columnset2:
        start = time.time()

        non_null_df = data.filter(col(col2).isNotNull())

        assembler = VectorAssembler(inputCols=columnset1, outputCol="features")
        non_null_df = assembler.transform(non_null_df)

        train_df, test_df = non_null_df.randomSplit([1.0 - test_ratio, test_ratio])

        dt = DecisionTreeRegressor(
            featuresCol="features",
            labelCol=col2
        )
        dt_model = dt.fit(train_df)

        model_path = f"{save_path}/{col2}_dt_model"
        dt_model.save(model_path)
        print(f"Model for column '{col2}' saved to {model_path}")

        predictions = dt_model.transform(test_df)
        evaluator = RegressionEvaluator(labelCol=col2, predictionCol="prediction", metricName="r2")
        r2 = evaluator.evaluate(predictions)
        finish = time.time()
        print(f'R2: {r2}', f'Time: {finish - start}')
        accuracies[col2] = r2
        print('=================================================================')

    return accuracies

# **Save column names which i used for predicting null values in columns.**

In [ ]:
train_colls = medians + ['emp_title_encoded','home_ownership_encoded','verification_status_encoded','loan_status_encoded','purpose_encoded','initial_list_status_encoded']

In [ ]:
with open('/content/drive/MyDrive/Bank Project/train_colls.pkl', 'wb') as file:
    pickle.dump(train_colls, file)

# **Create models and save them.**

In [ ]:
accuracies = save_models_without_filling(train, train_colls, models, '/content/drive/MyDrive/Bank Project/')

Model for column 'emp_length' saved to /content/drive/MyDrive/Bank Project//emp_length_dt_model
R2: 0.0924937828462874 Time: 251.5166997909546
Model for column 'mths_since_last_record' saved to /content/drive/MyDrive/Bank Project//mths_since_last_record_dt_model
R2: 0.28380138544666345 Time: 116.3836419582367
Model for column 'mths_since_last_major_derog' saved to /content/drive/MyDrive/Bank Project//mths_since_last_major_derog_dt_model
R2: 0.041513466788972386 Time: 133.5477545261383
Model for column 'tot_coll_amt' saved to /content/drive/MyDrive/Bank Project//tot_coll_amt_dt_model
R2: -0.01876434589677567 Time: 263.05756092071533
Model for column 'tot_cur_bal' saved to /content/drive/MyDrive/Bank Project//tot_cur_bal_dt_model
R2: 0.49984299354304895 Time: 266.2573194503784
Model for column 'open_acc_6m' saved to /content/drive/MyDrive/Bank Project//open_acc_6m_dt_model
R2: 1.9019958811150595e-05 Time: 199.76341342926025
Model for column 'open_act_il' saved to /content/drive/MyDrive/B

# **Choose columns which has models more than 40% r2.**

In [ ]:
model_replacement = ['tot_cur_bal', 'max_bal_bc', 'total_rev_hi_lim', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_rev_tl_op', 'mort_acc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pub_rec_bankruptcies', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'month_next_pymnt_d', 'year_next_pymnt_d']

In [ ]:
len(model_replacement + train_colls)

66

In [ ]:
with open('/content/drive/MyDrive/Bank Project/median_replacement.pkl', 'wb') as file:
    pickle.dump(medians, file)
with open('/content/drive/MyDrive/Bank Project/model_replacement.pkl', 'wb') as file:
    pickle.dump(model_replacement, file)
with open('/content/drive/MyDrive/Bank Project/train_colls.pkl', 'wb') as file:
    pickle.dump(train_colls, file)

# **Fill missing values.**

In [ ]:
def fill_missing_values(spark, spark_df):
    with open('/content/drive/MyDrive/Bank Project/median_replacement.pkl', 'rb') as file:
        median_replacement_columns = pickle.load(file)

    with open('/content/drive/MyDrive/Bank Project/model_replacement.pkl', 'rb') as file:
        model_replacement_columns = pickle.load(file)

    with open('/content/drive/MyDrive/Bank Project/train_colls.pkl', 'rb') as file:
        train_colls = pickle.load(file)
    relevant_columns = train_colls + ['acc_now_delinq', 'delinq_2yrs', 'delinq_amnt'] + model_replacement_columns
    spark_df = spark_df.select([col for col in spark_df.columns if col in relevant_columns])

    spark_df = spark_df.cache()

    median_parameters = spark.read.csv('/content/drive/MyDrive/Bank Project/train_parameters', header=True, inferSchema=True)

    median_dict = dict(zip(median_parameters.select('column_name').rdd.flatMap(lambda x: x).collect(),
                           median_parameters.select('median').rdd.flatMap(lambda x: x).collect()))
    for column in median_replacement_columns:
        if column in median_dict:
            median_value = median_dict[column]
            spark_df = spark_df.fillna(median_value, subset=[column])

    n = 1
    for column in model_replacement_columns:
        start_time = time.time()
        model_path = f'/content/drive/MyDrive/Bank Project/{column}_dt_model'
        model = DecisionTreeRegressionModel.load(model_path)

        assembler = VectorAssembler(inputCols=train_colls, outputCol='features')
        feature_df = assembler.transform(spark_df)

        predictions = model.transform(feature_df)

        predictions_dict = dict(predictions.select(column, 'prediction').rdd.map(lambda row: (row[column], row['prediction'])).collect())

        def replace_missing_value(value):
            return predictions_dict.get(value, None)

        replace_missing_value_udf = F.udf(replace_missing_value, DoubleType())

        spark_df = spark_df.withColumn(column, F.when(F.col(column).isNull(), replace_missing_value_udf(F.col(column))).otherwise(F.col(column)))
        end_time = time.time()
        print(f'{n} column processed in {end_time - start_time:.4f} seconds')
        n += 1

    spark_df.unpersist()

    return spark_df

In [ ]:
filled_missing_values = fill_missing_values(spark, train)

1 column processed in 130.5553 seconds
2 column processed in 33.7109 seconds
3 column processed in 34.8434 seconds
4 column processed in 34.4181 seconds
5 column processed in 31.2094 seconds
6 column processed in 32.1097 seconds
7 column processed in 29.7674 seconds
8 column processed in 30.7429 seconds
9 column processed in 29.2112 seconds
10 column processed in 29.4596 seconds
11 column processed in 29.2820 seconds
12 column processed in 28.3420 seconds
13 column processed in 28.2692 seconds
14 column processed in 29.6073 seconds
15 column processed in 28.5228 seconds
16 column processed in 29.1691 seconds
17 column processed in 29.7490 seconds
18 column processed in 29.1427 seconds
19 column processed in 30.1674 seconds
20 column processed in 30.9669 seconds
21 column processed in 32.1354 seconds
22 column processed in 30.2327 seconds
23 column processed in 30.4019 seconds
24 column processed in 29.5376 seconds
25 column processed in 30.7367 seconds


# **Visualization of each columns' situation in terms of null values.**

In [ ]:
calculate_nulls(filled_missing_values).show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----------+---+--------------+---------------+--------------+--------+-------+---------+----------+---------+---------+-------------+-----------+---------------+---------------+-------------+---------------+--------------------+-------------------+-----------+----------+----------------+-----------+--------------+-------+--------------------+--------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+--------------------+---------------+-----------------+--------------+--------------------------+--------------+-----------+-----------+----------------------+---------------------+-------------+------------+------------------+-----------------+------------------+-----------------+------------------------+-----------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|sub_grade|annual_inc|a

In [ ]:
filled_missing_values.show()

+---------+-----------+---------------+----+--------+-----------+---------+----------+----+--------------+---------------+--------------+--------+-------+---------+----------+---------+---------+-------------+----------------+---------------+---------------+-------------+---------------+--------------------+-------------------+-----------+------------------+----------------+-----------+--------------+-------+--------------------+--------+---------------------+--------------+-----------+---------+---------+-------------+-------------+-------------------+--------+--------------------+---------------+-----------------+--------------+--------------------------+--------------+-----------+-----------+-----------------+----------------------+---------------------------+-------------------+---------------+------------------+---------------------------+----------------------+---------------------+-------------+------------+------------------+-----------------+------------------+----------------

In [ ]:
filled_missing_values.write.parquet('/content/drive/MyDrive/Bank Project/filled_missing_values', mode = 'overwrite')

In [ ]:
filled_missing_values = spark.read.parquet('/content/drive/MyDrive/Bank Project/filled_missing_values')

# **Scale data.**

In [ ]:
def scale_data(spark, data, min_max_path):
    min_max_spark_df = spark.read.csv(min_max_path, header=True, inferSchema=True)

    column_name_col = min_max_spark_df.columns[0]
    cols_ = data.columns
    new_cols = cols_[:10] + cols_[11:] + [cols_[10]]
    data = data.select(new_cols)

    for column in data.columns:
        if column == 'acc_now_delinq':
            continue
        if column == 'delinq_2yrs':
            continue
        min_row = min_max_spark_df.filter(F.col(column_name_col) == column).select('min').first()
        max_row = min_max_spark_df.filter(F.col(column_name_col) == column).select('max').first()

        if min_row and max_row:
            min_val = min_row['min']
            max_val = max_row['max']

            if min_val == max_val:
                print(f"Column {column} has min_val == max_val == {min_val}. Filling this column with zeros.")
                data = data.withColumn(column, F.lit(0))
            else:
                data = data.withColumn(column, (F.col(column) - min_val) / (max_val - min_val))
        else:
            print(f"Min or Max value for column {column} not found in the min_max_df.")

    return data

In [ ]:
scaled_data = scale_data(spark, filled_missing_values, '/content/drive/MyDrive/Bank Project/train_parameters')

Min or Max value for column emp_title_encoded not found in the min_max_df.
Min or Max value for column home_ownership_encoded not found in the min_max_df.
Min or Max value for column verification_status_encoded not found in the min_max_df.
Min or Max value for column loan_status_encoded not found in the min_max_df.
Min or Max value for column purpose_encoded not found in the min_max_df.
Min or Max value for column initial_list_status_encoded not found in the min_max_df.


# **Round data.**

In [ ]:
def round_data(data):
    for column, dtype in data.dtypes:
        if isinstance(data.schema[column].dataType, NumericType):
            data = data.withColumn(column, F.round(F.col(column), 4))
        print(f'Column {column} processed')
    return data

In [ ]:
rounded_data = round_data(scaled_data)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

# **Value count of target variables.**

In [ ]:
rounded_data.groupBy('acc_now_delinq').count().show()
rounded_data.groupBy('delinq_2yrs').count().show()
rounded_data.groupBy('delinq_amnt').count().show()

+--------------+-------+
|acc_now_delinq|  count|
+--------------+-------+
|             1|   7119|
|             0|2332237|
+--------------+-------+

+-----------+-------+
|delinq_2yrs|  count|
+-----------+-------+
|          1| 279594|
|          3|  41166|
|          4|  15646|
|          2|  78483|
|          0|1924467|
+-----------+-------+

+-----------+-------+
|delinq_amnt|  count|
+-----------+-------+
|     0.0317|      1|
|     0.0124|      5|
|     0.1567|      1|
|     0.0287|      2|
|     0.0011|     87|
|     0.0261|      2|
|     0.0242|      2|
|      0.035|      1|
|      0.032|      1|
|     0.0331|      1|
|     0.0481|      1|
|        0.0|2333576|
|     0.1612|      1|
|      0.058|      1|
|     0.1718|      1|
|     0.0012|     65|
|      0.257|      1|
|     0.0472|      1|
|     0.0156|      9|
|     0.0347|      2|
+-----------+-------+
only showing top 20 rows



# **Create three different dataframe for each target variable**.

In [ ]:
main_delinq_amnt = rounded_data.select(*rounded_data.columns[:49] + rounded_data.columns[52:] + [rounded_data.columns[51]])
main_acc_now_delinq = rounded_data.select(*rounded_data.columns[:49] + rounded_data.columns[52:] + [rounded_data.columns[49]])
main_delinq_2yrs = rounded_data.select(*rounded_data.columns[:49] + rounded_data.columns[52:] + [rounded_data.columns[50]])

# **Preprocess first target variable: "delinq_amnt"**
---

### Determine minority and majority classes' amounts.

In [ ]:
majority_class_delinq_amnt = main_delinq_amnt.filter(col('delinq_amnt') == 0.0)
minority_classes_delinq_amnt = main_delinq_amnt.filter(col('delinq_amnt') != 0.0)

majority_class_count_delinq_amnt = majority_class_delinq_amnt.count()
minority_classes_count_delinq_amnt = minority_classes_delinq_amnt.count()

print(f"Majority class count: {majority_class_count_delinq_amnt}")
print(f"Minority classes count: {minority_classes_count_delinq_amnt}")

Majority class count: 2333576
Minority classes count: 5780


### Oversample minority class to majority.

In [ ]:
num_synthetic_samples_delinq_amnt = majority_class_count_delinq_amnt - minority_classes_count_delinq_amnt

fraction_to_sample_delinq_amnt = num_synthetic_samples_delinq_amnt / minority_classes_count_delinq_amnt

synthetic_df_delinq_amnt = minority_classes_delinq_amnt.sample(withReplacement=True, fraction=fraction_to_sample_delinq_amnt)

In [ ]:
balanced_delinq_amnt = majority_class_delinq_amnt.union(synthetic_df_delinq_amnt)
balanced_delinq_amnt.groupBy('delinq_amnt').count().orderBy('delinq_amnt').show()

+-----------+-------+
|delinq_amnt|  count|
+-----------+-------+
|        0.0|2333576|
|     1.0E-4| 136196|
|     2.0E-4| 164573|
|     3.0E-4| 227482|
|     4.0E-4| 118213|
|     5.0E-4|  70876|
|     6.0E-4|  65886|
|     7.0E-4|  53387|
|     8.0E-4|  42174|
|     9.0E-4|  40191|
|      0.001|  41130|
|     0.0011|  34889|
|     0.0012|  26197|
|     0.0013|  33534|
|     0.0014|  37091|
|     0.0015|  28021|
|     0.0016|  28077|
|     0.0017|  26060|
|     0.0018|  23980|
|     0.0019|  28595|
+-----------+-------+
only showing top 20 rows



### Round balanced data.

In [ ]:
balanced_delinq_amnt = round_data(balanced_delinq_amnt)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Calculation of whisker values for each column and saving.

In [ ]:
def calculate_whiskers(data, column):
    pd_data = data.select(column).toPandas()[column]

    q1 = pd_data.quantile(0.25)
    q3 = pd_data.quantile(0.75)
    iqr = q3 - q1
    lower_whisker = q1 - 1.5 * iqr
    upper_whisker = q3 + 1.5 * iqr

    return lower_whisker, upper_whisker

In [ ]:
whisk_dict_delinq_amnt = {'column': [], 'lower_whisker': [], 'upper_whisker': []}
for column in balanced_delinq_amnt.columns:
    wiskers = {}
    if isinstance(balanced_delinq_amnt.schema[column].dataType, VectorUDT):
        print(f'Skipping column {column} as it is of type VectorUDT')
        continue
    lower_whisker_delinq_amnt, upper_whisker_delinq_amnt = calculate_whiskers(balanced_delinq_amnt, column)
    whisk_dict_delinq_amnt['column'].append(column)
    whisk_dict_delinq_amnt['lower_whisker'].append(lower_whisker_delinq_amnt)
    whisk_dict_delinq_amnt['upper_whisker'].append(upper_whisker_delinq_amnt)
    print(f'Column {column} processed')
with open('/content/drive/MyDrive/Bank Project/whiskers_delinq_amnt.pkl', 'wb') as file:
    pickle.dump(whisk_dict_delinq_amnt, file)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Handle outliers and replace by median value of train.

In [ ]:
import pickle
from pyspark.sql.functions import col, when
from pyspark.ml.linalg import VectorUDT

def handle_outliers(data, whisker_path):
    with open(whisker_path, 'rb') as file:
        whiskers = pickle.load(file)

    train_parameters = spark.read.csv('/content/drive/MyDrive/Bank Project/train_parameters', header=True, inferSchema=True)

    for column in data.columns:
        if isinstance(data.schema[column].dataType, VectorUDT):
            print(f'Skipping column {column} as it is of type VectorUDT')
            continue
        outliers = True


        outliers = False
        median_value = train_parameters.filter(col('column_name') == column).select('median').collect()[0][0]
        min_value = train_parameters.filter(col('column_name') == column).select('min').collect()[0][0]
        max_value = train_parameters.filter(col('column_name') == column).select('max').collect()[0][0]
        scaled_median = (median_value - min_value) / (max_value - min_value)

        column_index = whiskers['column'].index(column)

        lower_whisker = whiskers['lower_whisker'][column_index]
        upper_whisker = whiskers['upper_whisker'][column_index]

        data = data.withColumn(
            column,
            when(col(column).isNull(), None)
            .when(col(column) <= lower_whisker, scaled_median)
            .when(col(column) >= upper_whisker, scaled_median)
            .otherwise(col(column))
        )
        print(f'Column {column} processed successfully')
    return data

In [ ]:
final_delinq_amnt = handle_outliers(balanced_delinq_amnt, '/content/drive/MyDrive/Bank Project/whiskers_delinq_amnt.pkl')

Column loan_amnt processed successfully
Column funded_amnt processed successfully
Column funded_amnt_inv processed successfully
Column term processed successfully
Column int_rate processed successfully
Column installment processed successfully
Column sub_grade processed successfully
Column annual_inc processed successfully
Column addr_state processed successfully
Column dti processed successfully
Column fico_range_high processed successfully
Column inq_last_6mths processed successfully
Column open_acc processed successfully
Column pub_rec processed successfully
Column revol_bal processed successfully
Column revol_util processed successfully
Column total_acc processed successfully
Column out_prncp processed successfully
Column out_prncp_inv processed successfully
Column total_pymnt processed successfully
Column total_pymnt_inv processed successfully
Column total_rec_prncp processed successfully
Column total_rec_int processed successfully
Column last_pymnt_amnt processed successfully
Col

In [ ]:
final_delinq_amnt.write.parquet('/content/drive/MyDrive/Bank Project/final_delinq_amnt_data', mode = 'overwrite')

# **Preprocess second target variable: "acc_now_delinq"**
---

### See the values of "acc_now_delinq" target.

In [ ]:
main_acc_now_delinq.groupBy('acc_now_delinq').count().show()

+--------------+-------+
|acc_now_delinq|  count|
+--------------+-------+
|             1|   7119|
|             0|2332237|
+--------------+-------+



### Determine minority and majority classes.

In [ ]:
majority_class_acc_now_delinq = main_acc_now_delinq.filter(col('acc_now_delinq') == 0.0)
minority_classes_acc_now_delinq = main_acc_now_delinq.filter(col('acc_now_delinq') == 1)

majority_class_count_acc_now_delinq = majority_class_acc_now_delinq.count()
minority_classes_count_acc_now_delinq = minority_classes_acc_now_delinq.count()

print(f"Majority class count: {majority_class_count_acc_now_delinq}")
print(f"Minority classes count: {minority_classes_count_acc_now_delinq}")

Majority class count: 2332237
Minority classes count: 7119


### Oversample minority class to majority.

In [ ]:
num_synthetic_samples_acc_now_delinq = majority_class_count_acc_now_delinq - minority_classes_count_acc_now_delinq

fraction_to_sample_acc_now_delinq = num_synthetic_samples_acc_now_delinq / minority_classes_count_acc_now_delinq

synthetic_df_acc_now_delinq = minority_classes_acc_now_delinq.sample(withReplacement=True, fraction=fraction_to_sample_acc_now_delinq)

In [ ]:
balanced_acc_now_delinq = majority_class_acc_now_delinq.union(synthetic_df_acc_now_delinq)
balanced_acc_now_delinq.groupBy('acc_now_delinq').count().orderBy('acc_now_delinq').show()

+--------------+-------+
|acc_now_delinq|  count|
+--------------+-------+
|             0|2332237|
|             1|2324567|
+--------------+-------+



### Round data.

In [ ]:
balanced_acc_now_delinq = round_data(balanced_acc_now_delinq)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Calculate whiskers for second target and save.

In [ ]:
whisk_dict_acc_now_delinq = {'column': [], 'lower_whisker': [], 'upper_whisker': []}
for column in balanced_acc_now_delinq.columns:
    wiskers = {}
    if isinstance(balanced_acc_now_delinq.schema[column].dataType, VectorUDT):
        print(f'Skipping column {column} as it is of type VectorUDT')
        continue
    lower_whisker_acc_now_delinq, upper_whisker_acc_now_delinq = calculate_whiskers(balanced_acc_now_delinq, column)
    whisk_dict_acc_now_delinq['column'].append(column)
    whisk_dict_acc_now_delinq['lower_whisker'].append(lower_whisker_acc_now_delinq)
    whisk_dict_acc_now_delinq['upper_whisker'].append(upper_whisker_acc_now_delinq)
    print(f'Column {column} processed')
with open('/content/drive/MyDrive/Bank Project/whiskers_acc_now_delinq.pkl', 'wb') as file:
    pickle.dump(whisk_dict_acc_now_delinq, file)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Handle outliers.

In [ ]:
final_acc_now_delinq = handle_outliers(balanced_acc_now_delinq, '/content/drive/MyDrive/Bank Project/whiskers_acc_now_delinq.pkl')

Column loan_amnt processed successfully
Column funded_amnt processed successfully
Column funded_amnt_inv processed successfully
Column term processed successfully
Column int_rate processed successfully
Column installment processed successfully
Column sub_grade processed successfully
Column annual_inc processed successfully
Column addr_state processed successfully
Column dti processed successfully
Column fico_range_high processed successfully
Column inq_last_6mths processed successfully
Column open_acc processed successfully
Column pub_rec processed successfully
Column revol_bal processed successfully
Column revol_util processed successfully
Column total_acc processed successfully
Column out_prncp processed successfully
Column out_prncp_inv processed successfully
Column total_pymnt processed successfully
Column total_pymnt_inv processed successfully
Column total_rec_prncp processed successfully
Column total_rec_int processed successfully
Column last_pymnt_amnt processed successfully
Col

In [ ]:
final_acc_now_delinq.write.parquet('/content/drive/MyDrive/Bank Project/final_acc_now_delinq_data', mode='overwrite')

# **Preprocess third target variable: "delinq_2yrs"**
---

### Value count of each class in target variable "delinq_2yrs".

In [ ]:
main_delinq_2yrs.groupBy('delinq_2yrs').count().show()

+-----------+-------+
|delinq_2yrs|  count|
+-----------+-------+
|          1| 279594|
|          3|  41166|
|          4|  15646|
|          2|  78483|
|          0|1924467|
+-----------+-------+



### Determine minority and majority classes.

In [ ]:
majority_class_delinq_2yrs = main_delinq_2yrs.filter(col('delinq_2yrs') == 0.0)
minority_classes_delinq_2yrs = main_delinq_2yrs.filter(col('delinq_2yrs') != 0.0)

majority_class_count_delinq_2yrs = majority_class_delinq_2yrs.count()
minority_classes_count_delinq_2yrs = minority_classes_delinq_2yrs.count()

print(f"Majority class count: {majority_class_count_delinq_2yrs}")
print(f"Minority classes count: {minority_classes_count_delinq_2yrs}")

Majority class count: 1924467
Minority classes count: 414889


### Oversample minority class to majoritydata from minority classes in target variable.

In [ ]:
majority_class_delinq_2yrs = main_delinq_2yrs.filter(col('delinq_2yrs') == 0.0)
minority_classes_delinq_2yrs_1 = main_delinq_2yrs.filter(col('delinq_2yrs') == 1)
minority_classes_delinq_2yrs_2 = main_delinq_2yrs.filter(col('delinq_2yrs') == 2)
minority_classes_delinq_2yrs_3 = main_delinq_2yrs.filter(col('delinq_2yrs') == 3)
minority_classes_delinq_2yrs_4 = main_delinq_2yrs.filter(col('delinq_2yrs') == 4)

In [ ]:
num_synthetic_samples_delinq_2yrs_1 = majority_class_count_delinq_2yrs - minority_classes_delinq_2yrs_1.count()
fraction_1 = num_synthetic_samples_delinq_2yrs_1 / minority_classes_delinq_2yrs_1.count()
synthetic_df_delinq_2yrs_1 = minority_classes_delinq_2yrs_1.sample(withReplacement=True, fraction=fraction_1)

num_synthetic_samples_delinq_2yrs_2 = majority_class_count_delinq_2yrs - minority_classes_delinq_2yrs_2.count()
fraction_2 = num_synthetic_samples_delinq_2yrs_2 / minority_classes_delinq_2yrs_2.count()
synthetic_df_delinq_2yrs_2 = minority_classes_delinq_2yrs_2.sample(withReplacement=True, fraction=fraction_2)


num_synthetic_samples_delinq_2yrs_3 = majority_class_count_delinq_2yrs - minority_classes_delinq_2yrs_3.count()
fraction_3 = num_synthetic_samples_delinq_2yrs_3 / minority_classes_delinq_2yrs_3.count()
synthetic_df_delinq_2yrs_3 = minority_classes_delinq_2yrs_3.sample(withReplacement=True, fraction=fraction_3)


num_synthetic_samples_delinq_2yrs_4 = majority_class_count_delinq_2yrs - minority_classes_delinq_2yrs_4.count()
fraction_4 = num_synthetic_samples_delinq_2yrs_4 / minority_classes_delinq_2yrs_4.count()
synthetic_df_delinq_2yrs_4 = minority_classes_delinq_2yrs_4.sample(withReplacement=True, fraction=fraction_4)

### Balance all classes in target variable.

In [ ]:
balanced_delinq_2yrs = majority_class_delinq_2yrs.\
union(synthetic_df_delinq_2yrs_1) \
.union(synthetic_df_delinq_2yrs_2) \
.union(synthetic_df_delinq_2yrs_3) \
.union(synthetic_df_delinq_2yrs_4)

In [ ]:
balanced_delinq_2yrs.groupBy('delinq_2yrs').count().show()

+-----------+-------+
|delinq_2yrs|  count|
+-----------+-------+
|          0|1924467|
|          1|1641275|
|          2|1846547|
|          3|1882620|
|          4|1909161|
+-----------+-------+



### Round data.

In [ ]:
balanced_delinq_2yrs = round_data(balanced_delinq_2yrs)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Calculate whiskers of each columns and save.

In [ ]:
whisk_dict_delinq_2yrs = {'column': [], 'lower_whisker': [], 'upper_whisker': []}
for column in balanced_delinq_2yrs.columns:
    wiskers = {}
    if isinstance(balanced_delinq_2yrs.schema[column].dataType, VectorUDT):
        print(f'Skipping column {column} as it is of type VectorUDT')
        continue
    lower_whisker_delinq_2yrs, upper_whisker_delinq_2yrs = calculate_whiskers(balanced_delinq_2yrs, column)
    whisk_dict_delinq_2yrs['column'].append(column)
    whisk_dict_delinq_2yrs['lower_whisker'].append(lower_whisker_delinq_2yrs)
    whisk_dict_delinq_2yrs['upper_whisker'].append(upper_whisker_delinq_2yrs)
    print(f'Column {column} processed')
with open('/content/drive/MyDrive/Bank Project/whiskers_delinq_2yrs.pkl', 'wb') as file:
    pickle.dump(whisk_dict_delinq_2yrs, file)

Column loan_amnt processed
Column funded_amnt processed
Column funded_amnt_inv processed
Column term processed
Column int_rate processed
Column installment processed
Column sub_grade processed
Column annual_inc processed
Column addr_state processed
Column dti processed
Column fico_range_high processed
Column inq_last_6mths processed
Column open_acc processed
Column pub_rec processed
Column revol_bal processed
Column revol_util processed
Column total_acc processed
Column out_prncp processed
Column out_prncp_inv processed
Column total_pymnt processed
Column total_pymnt_inv processed
Column total_rec_prncp processed
Column total_rec_int processed
Column last_pymnt_amnt processed
Column last_fico_range_high processed
Column last_fico_range_low processed
Column tot_cur_bal processed
Column max_bal_bc processed
Column total_rev_hi_lim processed
Column avg_cur_bal processed
Column bc_open_to_buy processed
Column bc_util processed
Column mo_sin_old_rev_tl_op processed
Column mort_acc processed

### Handle outliers.

In [ ]:
final_delinq_2yrs = handle_outliers(balanced_delinq_2yrs, '/content/drive/MyDrive/Bank Project/whiskers_delinq_2yrs.pkl')

Column loan_amnt processed successfully
Column funded_amnt processed successfully
Column funded_amnt_inv processed successfully
Column term processed successfully
Column int_rate processed successfully
Column installment processed successfully
Column sub_grade processed successfully
Column annual_inc processed successfully
Column addr_state processed successfully
Column dti processed successfully
Column fico_range_high processed successfully
Column inq_last_6mths processed successfully
Column open_acc processed successfully
Column pub_rec processed successfully
Column revol_bal processed successfully
Column revol_util processed successfully
Column total_acc processed successfully
Column out_prncp processed successfully
Column out_prncp_inv processed successfully
Column total_pymnt processed successfully
Column total_pymnt_inv processed successfully
Column total_rec_prncp processed successfully
Column total_rec_int processed successfully
Column last_pymnt_amnt processed successfully
Col

### Save data for target variable "delinq_2yrs".

In [ ]:
final_delinq_2yrs.write.parquet('/content/drive/MyDrive/Bank Project/final_final_delinq_2yrs_data', mode = 'overwrite')